# Physics 5070 Final Project

### Bejan Ghomashi and Michael Reh

## Introduction

The goal of this project was to build python modules to study quantum systems. Specifically, an eigenstate solver and time evolution module were developed in order to study phenomena such as Rabi oscillations and High Harmonic Generation. 

Rabi oscillations are of interest in many physical systems. A system with two or more bound states separated by finite energy can be made to oscillate between the states in the presence of an oscillatory driving field. An important example of this, explored in this project, is a quantum system perturbed by a laser. The frequency of the oscillating electric field in a laser pulse can be tuned to the energy separating a two-level system, causing the probability of finding the system in one state or the other to oscillate in time, which we can observe via the populations of the different states. Though in this project it will be explored more as a toy model, Rabi oscillations appear in many experiments in fields such as quantum optics and quantum computing, and the model could be extended for these specific applications as needed.

High harmonic generation (HHG) is a non-linear process in which a target (e.g. atomic gas) is excited by a low frequency laser of high intensity. The emitted radiation is composed of odd harmonics of the fundamental frequency $\omega_0$. This is usually described by the "three-step model": Tunneling, free motion in the electric field, and recombination. In this project, we use a potential that mimics hydrogen, with bound states of similar energy, to produce an HHG spectrum. One major difference in this toy potential is that hydrogen supports an infinite number of bound states were as our potentials only support a few or even one (this is due to the toy potential having finite size, while true hydrogen has an infinite depth).

## Eigenstate Solver

The eigenstate solver was built based on the Numerov method for solving the time-independent Schrodinger equation (TISE). This method is stronger than Runge-Kutta methods for solving the TISE due the the absence of the first derivative in the equation. A few different implementations were tested for this project, and can be found in the module *numerov.py*. The main workhorse in this module is the *solve_TISE_numerov* method, which uses the Numerov method to solve the TISE for a given potential at a given energy. All subsequent functions are built on this method. 

To find the actual bound states, a shooting method was employed originally, with a bisection method used to search for output wavefunctions meeting an endpoint boundary condition. Because this method was found to be unstable, especially for scattering states, methods involving the matching of left/right moving wavefunctions and even/odd parity states, were implemented which helped to improve the stability of finding solutions for the subsequent analysis. The primary interface functions that manage the underlying calculations are the *find_bound_states* and *find_bound_states_matching* methods.

## Time Evolution

The time evolution module was developed using the Crank-Nicolson method of solving partial differential equations. This method is unitary (and so conserves probability) and unconditionally stable, so we can take any time step that is small enough to observe the physics we are interested in. The algorithm solves individual time steps as follows:

$$\psi(t+\Delta t)=e^{-iH\Delta t}\psi(t)\\
    e^{iH\Delta t/2}\psi(t+\Delta t)=e^{-iH\Delta t/2}\psi(t)\\
    (1+iH\Delta t/2)\psi(t+\Delta t)=(1-iH\Delta t/2)\psi(t)\\
    \psi(t+\Delta t)=(1+iH\Delta t/2)^{-1}(1-iH\Delta t/2)\psi(t)$$

In practice, the matrix $(1+iH\Delta t/2)$ and its inverse are banded-tridiagonal (in 1D, which is a simplification made in this project), which allows for optimizations in the matrix inversion step. In practice, three different linear solvers were tested from SciPy--the general, sparse, and banded solvers--with increases in speed in order of listing. The Crank-Nicolson solver can be found in the module *crank_nicolson.py*, with the main interface function being *do_time_evolution_mask*, and *banded_solver* performing most of the calculations.